In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

creditInfo = pd.read_csv('german_credit_data1.csv')
creditInfo.head(10)

,id,Age,Sex,Job,Housing,Checking account,Credit amount,Duration
0,0,67,male,2,own,little,1169,6
1,3,45,male,2,free,little,7882,42
2,4,53,male,2,free,little,4870,24
3,11,24,female,2,rent,little,4308,48
4,13,60,male,1,own,little,1199,24
5,14,28,female,2,rent,little,1403,15
6,15,32,female,1,own,little,1282,24
7,17,25,male,2,own,little,8072,30
8,21,44,male,2,rent,little,2647,6
9,22,48,male,1,rent,little,2241,10


# Информация по кредитам граждан Германии
<dd> age - возраст</dd>
<dd> sex - пол</dd>
<dd> job - количество работ</dd>
<dd> housing - есть своя жил. площадь</dd>
<dd> Saving accounts - информаци о сберегательном счету.</dd>
<dd> Checking account - чековый счет</dd>
<dd> Credit amount - размер кредита</dd>
<dd> Duration - время на сколько кредит взят</dd>
<dd> Purpose - для каких нужд был взят кредит</dd>

Таргет: коэффициент размера кредита в зависимости от количество работ

In [179]:
creditInfo['credit_coef'] = creditInfo['Credit amount'] / creditInfo['Job']

Удалим размер кредита из колонки

In [180]:
creditInfo = creditInfo.drop(['Credit amount'], axis = 1)

Переведем sex и housing во множество {0, 1}.  Checking account в {0, 2}



In [181]:
creditInfo['Sex'] = creditInfo['Sex'].apply(lambda x: 0 if x == 'female' else 1)
creditInfo['Housing'] = creditInfo['Housing'].apply(lambda x: 0 if x == 'own' else 1)

accountMap = {'little': 0, 'moderate': 1, 'rich' : 2}
creditInfo['Checking account'] = creditInfo['Checking account'].apply(lambda x: accountMap[x])

print(creditInfo)

      id  Age  Sex  Job  Housing  Checking account  Duration  credit_coef
0      0   67    1    2        0                 0         6   584.500000
1      3   45    1    2        1                 0        42  3941.000000
2      4   53    1    2        1                 0        24  2435.000000
3     11   24    0    2        1                 0        48  2154.000000
4     13   60    1    1        0                 0        24  1199.000000
..   ...  ...  ...  ...      ...               ...       ...          ...
995  936   28    0    1        0                 2         9   745.000000
996  956   49    1    1        0                 2        30  3656.000000
997  975   57    0    1        0                 2        24  1258.000000
998  982   28    0    3        0                 2        21   974.333333
999  986   33    1    2        0                 2        42  3144.500000

[1000 rows x 8 columns]


Сохраним датасет с категориальными признаками для использования его в градиентном бустинге

In [182]:
creditInfo['id'] = [i for i in range(len(creditInfo))]
creditInfo = creditInfo.set_index('id')
creditInfo_for_lgb = creditInfo.copy()
creditInfo_for_lgb.head()

,Age,Sex,Job,Housing,Checking account,Duration,credit_coef
id,,,,,,,
0,67,1,2,0,0,6,584.5
1,45,1,2,1,0,42,3941.0
2,53,1,2,1,0,24,2435.0
3,24,0,2,1,0,48,2154.0
4,60,1,1,0,0,24,1199.0


In [183]:
creditInfo['Duration'].value_counts()

24    184
12    179
18    113
36     83
6      75
15     64
9      49
48     48
30     40
21     30
10     28
60     13
27     13
42     11
11      9
20      8
8       7
4       6
45      5
7       5
39      5
14      4
13      4
28      3
33      3
16      2
54      2
22      2
40      1
47      1
26      1
72      1
5       1
Name: Duration, dtype: int64

Реализуем mean-target encoding, заменив колонку Duration колонкой DurationCategory

In [184]:
creditInfo['Duration'] = creditInfo['Duration'].map(creditInfo.groupby(['Duration'])['credit_coef'].mean())
creditInfo = creditInfo.rename(columns={'Duration': 'DurationCategory'})
creditInfo.head(10)

,Age,Sex,Job,Housing,Checking account,DurationCategory,credit_coef
id,,,,,,,
0,67,1,2,0,0,1083.157778,584.5
1,45,1,2,1,0,3098.969697,3941.0
2,53,1,2,1,0,1720.275362,2435.0
3,24,0,2,1,0,3650.503472,2154.0
4,60,1,1,0,0,1720.275362,1199.0
5,28,0,2,1,0,1261.807292,701.5
6,32,0,1,0,0,1720.275362,1282.0
7,25,1,2,0,0,2473.187500,4036.0
8,44,1,2,1,0,1083.157778,1323.5


In [185]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = creditInfo.drop(['credit_coef'], axis=1)
y = creditInfo['credit_coef']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model=LinearRegression()
model.fit(X_train, y_train)
for column, coef in zip(X_train.columns, model.coef_):
    print(column, coef)
print(model.intercept_)

Age -1.7168425959285565
Sex 165.2836477308624
Job -625.7913185506463
Housing 302.6752222090431
Checking account 6.5421838600549025
DurationCategory 1.0488745071703818
969.140272167393


In [186]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

features = X_test
y_pred = model.predict(features) 
y_true = y_test

print(f"MSE = {mean_squared_error(y_true, y_pred)}")
print(f"MAE = {mean_absolute_error(y_pred, y_true)}")
print(f"r2_score  = {r2_score(y_true, y_pred)}")

MSE = 1430878.032519111
MAE = 823.2760896901251
r2_score  = 0.453268393558805


In [187]:
df = pd.DataFrame(y_true.copy())
df['prediction'] = y_pred
df.head(15)

,credit_coef,prediction
id,,
521,6141.333333,2872.289568
737,967.000000,982.986421
740,1309.000000,1836.434565
660,672.000000,1386.736034
411,3573.000000,1455.186803
678,388.000000,979.552735
626,2507.000000,1793.747120
513,2028.500000,1619.912208
859,1260.000000,766.302600


Моя модель предсказывает коэффицент размера кредита не очень хорошо. Коэффицент MAE довольно большой.

**Лабораторная 4**

In [172]:
import numpy as np
import lightgbm as lgb

creditInfo_for_lgb['Duration'] = creditInfo_for_lgb['Duration'].astype('category')
children_features = np.unique(creditInfo_for_lgb['Duration'].values, axis=0)

X = creditInfo_for_lgb.drop(['credit_coef'], axis=1)
y = creditInfo_for_lgb['credit_coef']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

model = lgb.LGBMRegressor(num_leaves=31, 
                          learning_rate=0.1, 
                          n_estimators=100,  
                          categorical_feature=children_features,  
                          random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print()
print(f"MSE: {np.mean((y_test - y_pred) ** 2)}")
print(f"MAE = {mean_absolute_error(y_pred, y_test)}")
print(f"r2_score  = {r2_score(y_true, y_pred)}")
print()


MSE: 1962742.23600486
MAE = 942.9175028621898
r2_score  = 0.25004564237267535



/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Можно сделать вывод, что по заданным данным тяжело предсказать коэффицент размера кредита.